# Assignment week 7

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Micha de Groot & Kaj Meijer

__Student id(s)__ : 10434410 & 10509534

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

[Selfie](http://imgur.com/RiMa4YY)

 
In this task we will try to expand the information that we find in the infoboxes using WordNet.

We follow the idea described in the YAGO paper: a string like `a beautiful actress from Denmark` is chopped up in a pre-modifier, a head and a post modifier. Here the head obviously is `actress`. 

## The tasks

 We perfom the task on a spreadsheet containing information about almost 1M persons found in the english Wikipedia.
In particular we are interested in the  `description` column.

1. (20%) For each string in the `description` column do the following:
    1. Determine pre-modifier,  head and  post modifier: store these in new columns
2.  (20%) For each head, determine the list of all ancestors of that head in WordNet up to "person": store this list in a new column.
3. (20%) Use Hearst patterns to extract something like the `description` from the `rdf-schema#comment`
    * Add your version as a new column
    * Design a comparison of your work with the description column which outputs something like accuracy/precision/recall. You are free, as long as you do it well.
4. (20%) Use Hearst patterns plus whatever else you like to extract the nationatility of the person described.
    * Include a brief evaluation.
    
2(actually 5). (20%) We are interested in gender aspects. Can you answer these questions:
    1. Can we determine the gender from the person from the description and its expansion? 
        * E.g., we could say it is a female if it was actress, but probably not say it is a male if it was actor
    2. There are two columns indicating the gender of the persons. One is made based on the first name, the other on the 3rd person pronouns in the longer description. Both are fairly accurate (more than .9 precision). Can you find a couple of remarkable differences between males and females based on your extracted wordnet concepts?

## Data
Available at <http://maartenmarx.nl/teaching/KennisSystemen/Data/PersonDF_naam_sex_description_snippet.csv.gz>
    
## Hints
* Do this in a python notebook, using pandas, and possibly the pattern module for parsing the description.
* Make a 10K version of the file and experiment with that first.

In [4]:

# read 

import pandas as pd
Pdf = pd.read_csv('http://maartenmarx.nl/teaching/KennisSystemen/Data/PersonDF_naam_sex_description_snippet.csv.gz',
                  compression='gzip')
Pdf = Pdf[Pdf.columns[2:]]
print(Pdf.shape)

Pdf.head()

(969996, 7)


,rdf-schema#label,description,gender,rdf-schema#comment,gender_from_pronouns,comment_length,word_comment_length
0,Ingeborg Moen Borgerud,Norwegian businesswoman and politician,U,Ingeborg Moen Borgerud (born 23 September 1949...,NaN,128.0,19
1,Ingeborg Nilsson,Figure skater,U,Ingeborg Nilsson (1924 – 1995) was a Norwegian...,female,247.0,41
2,Ingeborg Norell,Finnish heroine / life saver,U,Ingeborg Norell (born 1727) was the first Finn...,female,242.0,41
3,Ingeborg Pehrson,Danish actor,U,Ingeborg Pehrson (16 December 1886 - 11 April ...,NaN,88.0,16
4,Ingeborg Pfüller,Argentine shot putter and discus thrower,U,Ingeborg Ella Pfüller (born January 1 1932) wa...,female,301.0,52


In [5]:
newPdf = Pdf[0:1000]

Pdf = newPdf

print len(Pdf)

1000


In [6]:
# Opgave 1
from pattern.en import pprint, parse
import csv
import re 
import numpy as np


#with open('new2.csv', 'rb') as csvfile:
#    csvReader = csv.reader(csvfile, delimiter='\t')
    
    
column = Pdf['description']
premodList = []
headmodList = []
postmodList = []

for row in column:
    #print row
    #splitRow = row[0].split(',')[3]
    pre = []
    head = []
    post = []
    if pd.notnull(row):
        splitRow = parse(row).split(' ')
    else:
        premodList.append(pre)
        headmodList.append(head)
        postmodList.append(post)
        continue
    #print splitRow
    for word in splitRow:
        addWord = word.split('/')[0]
        if re.match('.*/NN(P)?/.*', word):
            #print word, "NN"
            head.append(addWord)
        if re.match('.*/JJ/.*', word):
            #print word, "JJ"
            pre.append(addWord)
        if not re.match('.*/O', word):
            #print word, "Not O"
            post.append(addWord)
    premodList.append(pre)
    headmodList.append(head)
    postmodList.append(post)
    


premodList = np.array(premodList)
headmodList = np.array(headmodList)
postmodList = np.array(postmodList)

print len(headmodList)
print len(premodList)
print len(postmodList)

# newPdf['head'] = heads.values
# in this simple example we add the number of tokens in the description as a new column
Pdf['premod'] = premodList
Pdf['head'] = headmodList
Pdf['postmod'] = postmodList
Pdf.head()

1000
1000
1000


,rdf-schema#label,description,gender,rdf-schema#comment,gender_from_pronouns,comment_length,word_comment_length,premod,head,postmod
0,Ingeborg Moen Borgerud,Norwegian businesswoman and politician,U,Ingeborg Moen Borgerud (born 23 September 1949...,NaN,128.0,19,[Norwegian],"[businesswoman, politician]",[]
1,Ingeborg Nilsson,Figure skater,U,Ingeborg Nilsson (1924 – 1995) was a Norwegian...,female,247.0,41,[],"[Figure, skater]",[]
2,Ingeborg Norell,Finnish heroine / life saver,U,Ingeborg Norell (born 1727) was the first Finn...,female,242.0,41,[Finnish],"[heroine, life, saver]",[]
3,Ingeborg Pehrson,Danish actor,U,Ingeborg Pehrson (16 December 1886 - 11 April ...,NaN,88.0,16,[Danish],[actor],[]
4,Ingeborg Pfüller,Argentine shot putter and discus thrower,U,Ingeborg Ella Pfüller (born January 1 1932) wa...,female,301.0,52,"[Argentine, shot]","[putter, discus, thrower]",[]


In [17]:
# Opgave 2
from nltk.corpus import wordnet as wn

heads = Pdf['head']

# Hier moet je nog een try-catch omheen zetten. In sommige gevallen is er namelijk geen head
hypernymList = []
for head in heads:
    localList = []
    try:
        #print head
        if pd.notnull(head).all():
            head = head[0]
            headwn = wn.synset(head+".n.01")
            #print headwn
            hypernym = headwn.hypernyms()
            #print hypernym
            while(str(hypernym) != "[Synset('person.n.01')]"):
                hypernym = hypernym[0].hypernyms()
                localList.append(hypernym)
                #print hypernym
    except: 
        pass
    hypernymList.append(localList)
    
hypernymList = np.array(hypernymList)

Pdf['Hypernym'] = hypernymList
Pdf.head()

,rdf-schema#label,description,gender,rdf-schema#comment,gender_from_pronouns,comment_length,word_comment_length,premod,head,postmod,Hypernym
0,Ingeborg Moen Borgerud,Norwegian businesswoman and politician,U,Ingeborg Moen Borgerud (born 23 September 1949...,NaN,128.0,19,[Norwegian],"[businesswoman, politician]",[],"[[Synset('capitalist.n.02')], [Synset('person...."
1,Ingeborg Nilsson,Figure skater,U,Ingeborg Nilsson (1924 – 1995) was a Norwegian...,female,247.0,41,[],"[Figure, skater]",[],"[[Synset('artwork.n.01')], [Synset('visual_com..."
2,Ingeborg Norell,Finnish heroine / life saver,U,Ingeborg Norell (born 1727) was the first Finn...,female,242.0,41,[Finnish],"[heroine, life, saver]",[],"[[Synset('portrayal.n.02')], [Synset('acting.n..."
3,Ingeborg Pehrson,Danish actor,U,Ingeborg Pehrson (16 December 1886 - 11 April ...,NaN,88.0,16,[Danish],[actor],[],"[[Synset('entertainer.n.01')], [Synset('person..."
4,Ingeborg Pfüller,Argentine shot putter and discus thrower,U,Ingeborg Ella Pfüller (born January 1 1932) wa...,female,301.0,52,"[Argentine, shot]","[putter, discus, thrower]",[],"[[Synset('player.n.01')], [Synset('contestant...."


In [33]:
# Opgave 3
from pattern.en import pprint, parse
import csv
import re 

'''
This code analyses the comment column of the csv file and 
looks for a Hears pattern of the form X is Y/X was Y.
After that it compares the found Y to the original description of the csv file.
For the comparison the F1 score is used. It is observed that 
the newley generated descriptions is almost always longer than
the original.
This results in a high recall and low precision on average.
Smoothing with the F1 score tis therefore always necessary.
'''

def hearstPattern(text):
    #text = text.split('.')[0]
    if re.match('.* was .*', text):
        text = re.split(" was ", text)
        if re.match('.* is .*', text[0]):
            text = re.split(" is ", text[0])
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
        else:
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
    elif re.match('.* is .*', text):
        text = re.split(" is ", text)
        text = text[1].split('.')[0]
        # print "Desc: ", text, "\n"
    return text

# For the comparison we use the F1 score. 
def compare(description1, description2):
    description1 = description1.lower().split()
    description2 = description2.lower().split()
    if( len(description1) == 0 or len(description2) == 0):
        return 0
    truepositives = 0.0
    for word in description1:
        if word in description2:
            truepositives +=1
    recall = truepositives / len(description1)
    precision = truepositives / len(description2)
    #print "Recall: ", recall
    #print "Precision: ", precision
    if precision == 0 and recall == 0:
        return 0
    
    F1score = 2*(precision*recall)/(precision+recall)
    return F1score

comments = Pdf['rdf-schema#comment']
description = Pdf['description']
similarityList = []
for index in enumerate(comments):
    originalDescription = description[index[0]] 
    #print originalDescription
    if pd.notnull(comments[index[0]]):
        pattern = hearstPattern(comments[index[0]])
    #print pattern
    if pd.notnull(originalDescription) and pd.notnull(comments[index[0]]):
        #print comments[index[0]]
        similarity = compare(originalDescription, pattern)
    else:
        similarity = 0
    #print similarity, "\n"
    similarityList.append(similarity)
    
similarityList = np.array(similarityList)

Pdf['similarity'] = similarityList
Pdf.head()

,rdf-schema#label,description,gender,rdf-schema#comment,gender_from_pronouns,comment_length,word_comment_length,premod,head,postmod,similarity
0,Ingeborg Moen Borgerud,Norwegian businesswoman and politician,U,Ingeborg Moen Borgerud (born 23 September 1949...,NaN,128.0,19,[Norwegian],"[businesswoman, politician]",[],0.400000
1,Ingeborg Nilsson,Figure skater,U,Ingeborg Nilsson (1924 – 1995) was a Norwegian...,female,247.0,41,[],"[Figure, skater]",[],0.666667
2,Ingeborg Norell,Finnish heroine / life saver,U,Ingeborg Norell (born 1727) was the first Finn...,female,242.0,41,[Finnish],"[heroine, life, saver]",[],0.080000
3,Ingeborg Pehrson,Danish actor,U,Ingeborg Pehrson (16 December 1886 - 11 April ...,NaN,88.0,16,[Danish],[actor],[],0.250000
4,Ingeborg Pfüller,Argentine shot putter and discus thrower,U,Ingeborg Ella Pfüller (born January 1 1932) wa...,female,301.0,52,"[Argentine, shot]","[putter, discus, thrower]",[],0.210526


In [37]:
# Opgave 4

# Opgave 3
from pattern.en import pprint, parse
import csv
import re 

'''
Looks in the comments for the first descriptive sentence. 
The observed pattern is that each sentence that contains
'was a(n)' or 'is a(n)' usually has the nationality in that 
sentence. It is assumed that the first capitalized
word is the nationality.
This works for almost all cases that actually contain a
nationality in the comment
'''

def hearstPattern(text):
    #text = text.split('.')[0]
    if re.match('.* was .*', text):
        text = re.split(" was ", text)
        if re.match('.* is .*', text[0]):
            text = re.split(" is ", text[0])
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
        else:
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
    elif re.match('.* is .*', text):
        text = re.split(" is ", text)
        text = text[1].split('.')[0]
        # print "Desc: ", text, "\n"
    return text

def findNationality(text):
    text = text.split()
    if text[0] == 'a' or text[0] == 'an':
        for i in range(len(text)):
            if text[i][0].isupper():
                return text[i]
    return None

nationalityList = []

comments = Pdf['rdf-schema#comment']
similarityList = []
for comment in comments:
    if pd.notnull(comment):
        pattern = hearstPattern(comment)
    #print "C: ", comment
    #print "P: ", pattern
        nationality = findNationality(pattern)
    #print "N: ", nationality, "\n"
    else:
        nationality = 0
    nationalityList.append(nationality)

nationalityList = np.array(nationalityList)
Pdf['Nationality'] = nationalityList
Pdf.head()



,rdf-schema#label,description,gender,rdf-schema#comment,gender_from_pronouns,comment_length,word_comment_length,premod,head,postmod,similarity,Nationality
0,Ingeborg Moen Borgerud,Norwegian businesswoman and politician,U,Ingeborg Moen Borgerud (born 23 September 1949...,NaN,128.0,19,[Norwegian],"[businesswoman, politician]",[],0.400000,Norwegian
1,Ingeborg Nilsson,Figure skater,U,Ingeborg Nilsson (1924 – 1995) was a Norwegian...,female,247.0,41,[],"[Figure, skater]",[],0.666667,Norwegian
2,Ingeborg Norell,Finnish heroine / life saver,U,Ingeborg Norell (born 1727) was the first Finn...,female,242.0,41,[Finnish],"[heroine, life, saver]",[],0.080000,None
3,Ingeborg Pehrson,Danish actor,U,Ingeborg Pehrson (16 December 1886 - 11 April ...,NaN,88.0,16,[Danish],[actor],[],0.250000,Danish
4,Ingeborg Pfüller,Argentine shot putter and discus thrower,U,Ingeborg Ella Pfüller (born January 1 1932) wa...,female,301.0,52,"[Argentine, shot]","[putter, discus, thrower]",[],0.210526,Argentina


Opgave 5.1

It is possible to detemine the gender of the person from the description if it contains gender specific terms. Most of the gender specific terms are female terms like actress, where actor could be male of female. 

Opgave 5.2

!!! We need data for this. !!!

In [ ]:
# http://www.clips.ua.ac.be/pages/pattern-en#parser
>>> from pattern.en import parse
>>> print parse('a beautiful actress from Denmark walks')

In [ ]:
from pattern.en import pprint, parse
pprint(parse('a beautiful actress from Denmark walks', lemmata=True))

In [ ]:
pprint(parse('Ingeborg Moen Borgerud  is a Norwegian lawyer businessperson and former politician for the Labour Party.'))

In [ ]:
# read 

import pandas as pd
Pdf = pd.read_csv('http://maartenmarx.nl/teaching/KennisSystemen/Data/PersonDF_naam_sex_description_snippet.csv.gz',
                 compression='gzip')
Pdf = Pdf[Pdf.columns[2:]]
print Pdf.shape

Pdf.head()

In [ ]:
# add some columns to Pdf 

# in this simple example we add the number of tokens in the description as a new column

def telwoordjes(string):
    return len(string.split())

Pdf['Description_length']= Pdf.description.astype(str).apply(telwoordjes)

Pdf.Description_length.head()

In [ ]:
# save, gzip, and load again

%time Pdf.to_pickle('PersonDF_naam_sex_description_snippetEXTRA.pkl')

%ls -lh 'PersonDF_naam_sex_description_snippetEXTRA.pkl'
%time !gzip PersonDF_naam_sex_description_snippetEXTRA.pkl
%ls -lh PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
%time !gunzip PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
# read 

%time Pdf = pd.read_pickle('PersonDF_naam_sex_description_snippetEXTRA.pkl')

!rm PersonDF_naam_sex_description_snippetEXTRA.pkl

In [ ]:
# inspect the full descriptions

for x in Pdf['rdf-schema#comment'][:10]:
    print x